# Road Trip Optimization

Determine the optimal driving route between up to 70 points of interest.

*Based on Randal Olson's notebook, [Computing the optimal road trip across the U.S.](https://github.com/rhiever/Data-Analysis-and-Machine-Learning-Projects/blob/master/optimal-road-trip/Computing%20the%20optimal%20road%20trip%20across%20the%20U.S..ipynb). All credit for the genetic algorithm code and the query Google Maps API code goes to him with a few adjustments made by me. See [data_collection.py](./src/data_collection.py) and [genetic_algorithm.py](./src/genetic_algorithm.py) for more details.*

## Enter Your Trip Information

In [1]:
# Enter your points of interest in the list below (include your starting location)
MY_POINTS_OF_INTEREST = [
    "San Francisco, California", # Starting Location
    "Yosemite Valley Visitor Center, Village Drive, Yosemite Valley, CA", # Yosemite NP
    "D L Bliss State Park, California 89, South Lake Tahoe, CA", # Lake Tahoe
    "Zion National Park Visitor Center, Zion – Mount Carmel Highway, Hurricane, UT", # Zion NP
    "Arches National Park Visitor Center, Moab, UT", # Arches NP
    "Monument Valley Navajo Tribal Park, Main Monument Valley Road, Oljato-Monument Valley, AZ", # Monument Valley
    "Island in the Sky Visitor Center, Grand View Point Road, Moab, UT", # Canyonlands NP
    "Bryce Canyon National Park Visitor Center, Utah 63, Bryce, UT", # Bryce Canyon NP
    "Moro Rock Trail, California", # Sequoia NP
    "Horseshoe Bend Parking Lot, Page, AZ", # Horseshoe Bend Trail
    "Grand Canyon Visitor Center, South Entrance Road, Grand Canyon Village, AZ", # Grand Canyon NP
    "Calf Creek Campground, Boulder, UT", # Grand Staircase-Escalante NM
    "Red Cliffs Recreation Area, Unnamed Road, Washington, UT", # Red Cliffs Recreation Nature Trail
    "Natural Bridges Visitor Center, Natural Bridge, Lake Powell, UT", # Natural Bridges NM
    "Kanarra Creek Trailhead, Kanarraville, UT", # Kanarra Creek Canyon Trail
    "San Simeon, CA", # South Big Sur Drive
    "Big Sur, CA", # Middle Big Sur Drive
    "Carmel-by-the-Sea, CA", # North Big Sur Drive
    "Castle Rock Entrance Station Parking Lot, Unnamed Road, Saratoga, CA", # Saratoga Gap Trail
]

# Enter your trip name (a unique identifier for your specified points of interest)
TRIP_NAME = "east"

## Collect Driving Distance/Duration Information

In [2]:
from src.data_collection import *
from config import GOOGLE_MAPS_API_KEY

# Determine distance and duration filename based on specified trip name
distance_duration_filename = "data/my_{}_points_of_interest_distance_duration.csv".format(TRIP_NAME)

# Try to create a distance and duration df containing all my points of interest from the filename
try:
    
    distance_duration_df = pd.read_csv(distance_duration_filename, index_col=0)

    # Create list of unique points of interest from df
    df_points_of_interest = set(pd.unique(distance_duration_df[['Venue 1', 'Venue 2']].values.ravel('K')))

    # Check if missing one or more of my points of interest in df
    if not set(MY_POINTS_OF_INTEREST).issubset(df_points_of_interest):
        
        raise Exception("Missing one or more of my points of interest in '{}'".format(distance_duration_filename))

# Create a distance and duration df with all my points of interest and save to the specified filename
except (FileNotFoundError, Exception) as e:
        
    # Query Google Maps API for one-way driving distances and durations
    distance_duration_data = query_gmaps_api_for_one_way_driving_distance_and_duration(MY_POINTS_OF_INTEREST, GOOGLE_MAPS_API_KEY)

    # Create DataFrame of one-way distances and durations
    distance_duration_df = create_distance_and_duration_df(distance_duration_data)

    # Save DataFrame to CSV
    distance_duration_df.to_csv(distance_duration_filename)
    
# Preview distance and duration df
distance_duration_df.head().sort_values('Distance (mi)', ascending=False)

,Venue 1,Venue 2,Distance (mi),Duration (s),Duration (hhmm)
3,"San Francisco, California","Arches National Park Visitor Center, Moab, UT",963,51219,14:13
4,"San Francisco, California","Monument Valley Navajo Tribal Park, Main Monum...",941,52195,14:29
2,"San Francisco, California","Zion National Park Visitor Center, Zion – Moun...",727,40019,11:06
1,"San Francisco, California","D L Bliss State Park, California 89, South Lak...",197,12542,3:29
0,"San Francisco, California","Yosemite Valley Visitor Center, Village Drive,...",191,14181,3:56


### *Optional : Display Full Name Squareform Distance/Duration Matrices*

In [3]:
# Add reverse travel information (B to A not just A to B) to distance and duration df
_df = add_reverse_travel_information_to_distance_duration_df(distance_duration_df)

# Create squareform matrices
distance_matrix = _df.pivot(index='Venue 1', columns='Venue 2', values='Distance (mi)').fillna(0).astype(int)
duration_matrix = _df.pivot(index='Venue 1', columns='Venue 2', values='Duration (s)').fillna(0).astype(int)
duration_matrix_hhmm = _df.pivot(index='Venue 1', columns='Venue 2', values='Duration (hhmm)').fillna("0:00")

In [4]:
# # Display distance matrix
# display(distance_matrix)

In [5]:
# # Display duration matrix
# display(duration_matrix)

In [6]:
# Display duration hhmm matrix
display(duration_matrix_hhmm)

Venue 2,"Arches National Park Visitor Center, Moab, UT","Big Sur, CA","Bryce Canyon National Park Visitor Center, Utah 63, Bryce, UT","Calf Creek Campground, Boulder, UT","Carmel-by-the-Sea, CA","Castle Rock Entrance Station Parking Lot, Unnamed Road, Saratoga, CA","D L Bliss State Park, California 89, South Lake Tahoe, CA","Grand Canyon Visitor Center, South Entrance Road, Grand Canyon Village, AZ","Horseshoe Bend Parking Lot, Page, AZ","Island in the Sky Visitor Center, Grand View Point Road, Moab, UT","Kanarra Creek Trailhead, Kanarraville, UT","Monument Valley Navajo Tribal Park, Main Monument Valley Road, Oljato-Monument Valley, AZ","Moro Rock Trail, California","Natural Bridges Visitor Center, Natural Bridge, Lake Powell, UT","Red Cliffs Recreation Area, Unnamed Road, Washington, UT","San Francisco, California","San Simeon, CA","Yosemite Valley Visitor Center, Village Drive, Yosemite Valley, CA","Zion National Park Visitor Center, Zion – Mount Carmel Highway, Hurricane, UT"
Venue 1,,,,,,,,,,,,,,,,,,,
"Arches National Park Visitor Center, Moab, UT",0:00,15:20,4:13,3:38,14:22,14:47,12:01,5:43,4:40,0:36,4:19,2:48,13:23,2:14,4:39,14:13,13:11,12:23,5:09
"Big Sur, CA",15:20,0:00,12:48,13:54,1:03,2:20,5:56,12:20,13:12,15:37,11:10,14:38,5:37,15:57,10:51,2:57,2:21,4:57,11:28
"Bryce Canyon National Park Visitor Center, Utah 63, Bryce, UT",4:13,12:48,0:00,1:21,11:51,12:15,10:30,5:05,2:44,4:30,1:47,4:41,10:51,4:41,2:08,12:28,10:39,10:03,1:51
"Calf Creek Campground, Boulder, UT",3:38,13:54,1:21,0:00,12:57,13:21,11:36,6:07,3:50,3:54,2:53,4:50,11:56,3:40,3:14,13:34,11:45,11:09,2:57
"Carmel-by-the-Sea, CA",14:22,1:03,11:51,12:57,0:00,1:25,4:59,11:22,12:15,14:39,10:13,13:40,4:39,14:59,9:53,2:00,2:22,4:00,10:31
"Castle Rock Entrance Station Parking Lot, Unnamed Road, Saratoga, CA",14:47,2:20,12:15,13:21,1:25,0:00,4:21,11:46,12:39,15:03,10:37,14:05,4:50,15:23,10:18,1:09,3:23,4:11,10:55
"D L Bliss State Park, California 89, South Lake Tahoe, CA",12:01,5:56,10:30,11:36,4:59,4:21,0:00,12:02,11:36,12:18,9:06,13:34,6:38,13:30,9:15,3:29,6:42,4:36,9:53
"Grand Canyon Visitor Center, South Entrance Road, Grand Canyon Village, AZ",5:43,12:20,5:05,6:07,11:22,11:46,12:02,0:00,2:24,6:12,4:54,3:07,10:23,4:20,4:38,12:01,10:11,11:22,4:39
"Horseshoe Bend Parking Lot, Page, AZ",4:40,13:12,2:44,3:50,12:15,12:39,11:36,2:24,0:00,5:08,2:50,2:04,11:12,3:21,2:34,12:50,11:03,11:08,2:18


### *Optional : Display Integer Name Squareform Distance/Duration Matrices*

In [7]:
# Add reverse travel information (B to A not just A to B) to distance and duration df
_df = add_reverse_travel_information_to_distance_duration_df(distance_duration_df)

# Convert venue columns to categorical type and create categorical code columns
_df['Venue 1'] = _df['Venue 1'].astype('category')
_df['Venue 1 Codes'] = _df['Venue 1'].cat.codes
_df['Venue 2'] = pd.Categorical(_df['Venue 2'], categories=_df['Venue 1'].cat.categories)
_df['Venue 2 Codes'] = _df['Venue 2'].cat.codes

# Create squareform matrices with codes
distance_matrix = _df.pivot(index='Venue 1 Codes', columns='Venue 2 Codes', values='Distance (mi)').fillna(0).astype(int)
duration_matrix = _df.pivot(index='Venue 1 Codes', columns='Venue 2 Codes', values='Duration (s)').fillna(0).astype(int)
duration_matrix_hhmm = _df.pivot(index='Venue 1 Codes', columns='Venue 2 Codes', values='Duration (hhmm)').fillna("0:00")

In [8]:
# # Preview new columns
# _df.sample(5)

In [9]:
# # Display distance matrix with code mappings
# display(distance_matrix)

# # Print dict of code: cat mappings for reference
# _ = dict(enumerate(_df['Venue 1'].cat.categories))
# for k, v in _.items():
#     print(k, ":", v)

In [10]:
# # Display duration matrix with code mappings
# display(duration_matrix)

# # Print dict of code: cat mappings for reference
# _ = dict(enumerate(_df['Venue 1'].cat.categories))
# for k, v in _.items():
#     print(k, ":", v)

In [11]:
# Display duration matrix hhmm with code mappings
display(duration_matrix_hhmm)

# Print dict of code: cat mappings for reference
_ = dict(enumerate(_df['Venue 1'].cat.categories))
for k, v in _.items():
    print(k, ":", v)

Venue 2 Codes,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
Venue 1 Codes,,,,,,,,,,,,,,,,,,,
0,0:00,15:20,4:13,3:38,14:22,14:47,12:01,5:43,4:40,0:36,4:19,2:48,13:23,2:14,4:39,14:13,13:11,12:23,5:09
1,15:20,0:00,12:48,13:54,1:03,2:20,5:56,12:20,13:12,15:37,11:10,14:38,5:37,15:57,10:51,2:57,2:21,4:57,11:28
2,4:13,12:48,0:00,1:21,11:51,12:15,10:30,5:05,2:44,4:30,1:47,4:41,10:51,4:41,2:08,12:28,10:39,10:03,1:51
3,3:38,13:54,1:21,0:00,12:57,13:21,11:36,6:07,3:50,3:54,2:53,4:50,11:56,3:40,3:14,13:34,11:45,11:09,2:57
4,14:22,1:03,11:51,12:57,0:00,1:25,4:59,11:22,12:15,14:39,10:13,13:40,4:39,14:59,9:53,2:00,2:22,4:00,10:31
5,14:47,2:20,12:15,13:21,1:25,0:00,4:21,11:46,12:39,15:03,10:37,14:05,4:50,15:23,10:18,1:09,3:23,4:11,10:55
6,12:01,5:56,10:30,11:36,4:59,4:21,0:00,12:02,11:36,12:18,9:06,13:34,6:38,13:30,9:15,3:29,6:42,4:36,9:53
7,5:43,12:20,5:05,6:07,11:22,11:46,12:02,0:00,2:24,6:12,4:54,3:07,10:23,4:20,4:38,12:01,10:11,11:22,4:39
8,4:40,13:12,2:44,3:50,12:15,12:39,11:36,2:24,0:00,5:08,2:50,2:04,11:12,3:21,2:34,12:50,11:03,11:08,2:18


0 : Arches National Park Visitor Center, Moab, UT
1 : Big Sur, CA
2 : Bryce Canyon National Park Visitor Center, Utah 63, Bryce, UT
3 : Calf Creek Campground, Boulder, UT
4 : Carmel-by-the-Sea, CA
5 : Castle Rock Entrance Station Parking Lot, Unnamed Road, Saratoga, CA
6 : D L Bliss State Park, California 89, South Lake Tahoe, CA
7 : Grand Canyon Visitor Center, South Entrance Road, Grand Canyon Village, AZ
8 : Horseshoe Bend Parking Lot, Page, AZ
9 : Island in the Sky Visitor Center, Grand View Point Road, Moab, UT
10 : Kanarra Creek Trailhead, Kanarraville, UT
11 : Monument Valley Navajo Tribal Park, Main Monument Valley Road, Oljato-Monument Valley, AZ
12 : Moro Rock Trail, California
13 : Natural Bridges Visitor Center, Natural Bridge, Lake Powell, UT
14 : Red Cliffs Recreation Area, Unnamed Road, Washington, UT
15 : San Francisco, California
16 : San Simeon, CA
17 : Yosemite Valley Visitor Center, Village Drive, Yosemite Valley, CA
18 : Zion National Park Visitor Center, Zion – Mo

## Determine Optimal Driving Route

In [17]:
from src.genetic_algorithm import *

# Run the genetic algorithm
results = run_genetic_algorithm(MY_POINTS_OF_INTEREST, distance_duration_filename, generations=5000, population_size=100)

# Update HTML file to display correct results
update_results_html_file("./src/results_template.html", "./tmp/results.html", results)

Generation 0 best: 7214 | Unique genomes: 100
('Island in the Sky Visitor Center, Grand View Point Road, Moab, UT', 'Grand Canyon Visitor Center, South Entrance Road, Grand Canyon Village, AZ', 'Calf Creek Campground, Boulder, UT', 'Red Cliffs Recreation Area, Unnamed Road, Washington, UT', 'Kanarra Creek Trailhead, Kanarraville, UT', 'Moro Rock Trail, California', 'San Simeon, CA', 'Carmel-by-the-Sea, CA', 'D L Bliss State Park, California 89, South Lake Tahoe, CA', 'Yosemite Valley Visitor Center, Village Drive, Yosemite Valley, CA', 'Zion National Park Visitor Center, Zion – Mount Carmel Highway, Hurricane, UT', 'Monument Valley Navajo Tribal Park, Main Monument Valley Road, Oljato-Monument Valley, AZ', 'Natural Bridges Visitor Center, Natural Bridge, Lake Powell, UT', 'Big Sur, CA', 'Horseshoe Bend Parking Lot, Page, AZ', 'Castle Rock Entrance Station Parking Lot, Unnamed Road, Saratoga, CA', 'San Francisco, California', 'Arches National Park Visitor Center, Moab, UT', 'Bryce Canyo

Generation 4000 best: 2915 | Unique genomes: 79
('Moro Rock Trail, California', 'Grand Canyon Visitor Center, South Entrance Road, Grand Canyon Village, AZ', 'Horseshoe Bend Parking Lot, Page, AZ', 'Monument Valley Navajo Tribal Park, Main Monument Valley Road, Oljato-Monument Valley, AZ', 'Natural Bridges Visitor Center, Natural Bridge, Lake Powell, UT', 'Arches National Park Visitor Center, Moab, UT', 'Island in the Sky Visitor Center, Grand View Point Road, Moab, UT', 'Calf Creek Campground, Boulder, UT', 'Bryce Canyon National Park Visitor Center, Utah 63, Bryce, UT', 'Zion National Park Visitor Center, Zion – Mount Carmel Highway, Hurricane, UT', 'Kanarra Creek Trailhead, Kanarraville, UT', 'Red Cliffs Recreation Area, Unnamed Road, Washington, UT', 'San Simeon, CA', 'Big Sur, CA', 'Carmel-by-the-Sea, CA', 'Castle Rock Entrance Station Parking Lot, Unnamed Road, Saratoga, CA', 'San Francisco, California', 'D L Bliss State Park, California 89, South Lake Tahoe, CA', 'Yosemite Valle

In [18]:
# View map of results (map opens in new tab)
!open ./tmp/results.html

In [19]:
# View distance and duration df of results
results_df = create_results_df(distance_duration_filename, results)
print("Total driving distance {} miles ...".format(results_df['Distance (mi)'].sum()))
display(results_df)

Total driving distance 2915 miles ...


,Venue 1,Venue 2,Duration (hhmm),Distance (mi)
0,"Moro Rock Trail, California","Grand Canyon Visitor Center, South Entrance Ro...",10:23,619
1,"Grand Canyon Visitor Center, South Entrance Ro...","Horseshoe Bend Parking Lot, Page, AZ",2:24,132
2,"Horseshoe Bend Parking Lot, Page, AZ","Monument Valley Navajo Tribal Park, Main Monum...",2:04,127
3,"Monument Valley Navajo Tribal Park, Main Monum...","Natural Bridges Visitor Center, Natural Bridge...",1:23,66
4,"Natural Bridges Visitor Center, Natural Bridge...","Arches National Park Visitor Center, Moab, UT",2:14,119
5,"Arches National Park Visitor Center, Moab, UT","Island in the Sky Visitor Center, Grand View P...",0:36,29
6,"Island in the Sky Visitor Center, Grand View P...","Calf Creek Campground, Boulder, UT",3:54,213
7,"Calf Creek Campground, Boulder, UT","Bryce Canyon National Park Visitor Center, Uta...",1:21,66
8,"Bryce Canyon National Park Visitor Center, Uta...","Zion National Park Visitor Center, Zion – Moun...",1:51,85
9,"Zion National Park Visitor Center, Zion – Moun...","Kanarra Creek Trailhead, Kanarraville, UT",0:58,46


### *Optional : Save Results Map and DataFrame Shown Above*

In [20]:
import os
import uuid

# Determine unique results identifier as tripName_resultsTotalDistance_threeCharUniqueIdentifier
unique_results_identifier = "{}_{}_{}".format(TRIP_NAME, str(results_df['Distance (mi)'].sum()), str(uuid.uuid4())[-3:])

# Save results map HTML
os.rename("./tmp/results.html", "./results/{}_results_map.html".format(unique_results_identifier))

# Save results df to CSV
results_df.to_csv("./results/{}_results_distance_duration.csv".format(unique_results_identifier))

## Cost Calculator

In [21]:
miles = 2922 - 2915
vehicle_miles_per_gallon = 30
price_per_gallon = 3.25

cost = (miles / vehicle_miles_per_gallon) * price_per_gallon

print("It costs ${:,.2f} to travel {} miles.".format(cost, miles))

It costs $0.76 to travel 7 miles.
